### List of tables:


1. [Table 3: Regressing the probability of player X to choose the selfish option, excluding observations](#quantity)

### List of figures:

2. [Figure 3: Player X’s beliefs](#belief)

#### Imports libraries

In [1]:
import matplotlib.pyplot as plt #plotting
import os #file system handling
import pandas as pd #dataframe handling
import statsmodels.api as sm #regression analysis
import statsmodels.formula.api as smf # regressions analysis

from matplotlib.ticker import FuncFormatter #formating graphs 
from scipy import stats #statistics module
from statsmodels.iolib.summary2 import summary_col #regression output table
from statsmodels.stats.anova import anova_lm #regression anova table
from statsmodels.stats.multicomp import MultiComparison #multi-comparisson tests 

#### Set project directory

In [2]:
PROJECT_FOLDER = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
FINAL_DATA_FOLDER = os.path.join(PROJECT_FOLDER, 'data', 'final')
TABLES_FOLDER = os.path.join(PROJECT_FOLDER, 'reports', 'tables')
FIGURES_FOLDER = os.path.join(PROJECT_FOLDER, 'reports', 'figures')

#### Set display format

In [3]:
pd.options.display.float_format = '{:.3f}'.format

#### Set plotting style

In [4]:
plt.style.use('classic')

#### Set plotting properties 

In [ ]:
bar_kwargs = dict(kind='bar', color='0.4', alpha=0.8, lw=0.5, width=0.7)
line_kwargs = dict(kind='line', lw=1, alpha=1, legend=True)
tick_kwargs = dict(size=5, which='both', right=False, top=False, direction='out', labelbottom=True)
font_kwargs = dict(fontsize=11, color='k')
grid_kwargs = dict(linewidth=1, axis="y", zorder=2, antialiased=True)
xlabel_kwargs = dict(fontsize=11, labelpad=3)
ylabel_kwargs = dict(fontsize=11, labelpad=3)
e_kwargs = dict(elinewidth=2, ecolor='0.15')
legend_kwargs = dict(frameon=False)

#### Retrieving dataframe

In [5]:
DATA = os.path.join(FINAL_DATA_FOLDER, 'fair_advice_final.feather')
df = pd.read_feather(DATA)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3598 entries, 0 to 3597
Data columns (total 24 columns):
Dataset                     3598 non-null int64
SubmitTime                  3598 non-null datetime64[ns]
Duration                    3598 non-null int64
Controlquestion             1799 non-null float64
Controlquestion_attempts    1799 non-null float64
Treatment                   3598 non-null int64
Treatment_name              3598 non-null object
Dictator                    3598 non-null int64
Decision                    3598 non-null int64
Advise                      3598 non-null int64
Follow                      1799 non-null float64
Belief1                     598 non-null float64
Belief2                     598 non-null float64
Age                         3598 non-null int64
Female                      3598 non-null int64
Education                   3598 non-null int64
Political                   3598 non-null int64
Trust_requester             1799 non-null float64
Motivatio